In [1]:
import torch
import clip
from PIL import Image
import numpy as np
import glob
import math
from scipy import spatial
import os

/home/toonies/anaconda3/envs/UTE-AI/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def calculate_cosine_distance(a, b):
    cosine_distance = float(spatial.distance.cosine(a, b))
    return cosine_distance

def calculate_cosine_similarity(a, b):
    cosine_similarity = 1 - calculate_cosine_distance(a, b)
    return cosine_similarity

In [6]:
device = "cpu"
print(device)
model, preprocess = clip.load("ViT-B/16", device=device)

cpu


In [8]:
des_path =  "data/CLIP_Features/"
paths = "data/KeyFramesC00_V00/"

video_paths = sorted(glob.glob(f"{paths}/*/"))
video_paths = ['/'.join(i.split('/')[:-1]) for i in video_paths]

# os.makedirs(des_path, exist_ok=True)

for vd_path in video_paths:
  print(vd_path)
  # check_file = int(vd_path.split('/')[-1].replace('C02_V',''))
  # if check_file <= 349:
  #   print(f"Skip: {vd_path}")
  #   continue

  re_feats = []
  keyframe_paths = glob.glob(f'{vd_path}/*.jpg')
  keyframe_paths = sorted(keyframe_paths, key=lambda x : x.split('/')[-1].replace('.jpg',''))

  for keyframe_path in keyframe_paths:
    image = preprocess(Image.open(keyframe_path)).unsqueeze(0).to(device)

    with torch.no_grad():
        image_feats = model.encode_image(image)

    image_feats /= image_feats.norm(dim=-1, keepdim=True)
    image_feats = image_feats.detach().cpu().numpy().astype(np.float16).flatten() 

    re_feats.append(image_feats)

  name_npy = vd_path.split('/')[-1]
  outfile = f'{des_path}/{name_npy}.npy'
  np.save(outfile, re_feats)

data/KeyFramesC00_V00/C00_V0000
data/KeyFramesC00_V00/C00_V0001


KeyboardInterrupt: 